In [7]:
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import math
import time
import multiprocessing
from multiprocessing import Pool

In [8]:
# cpu_count=multiprocessing.cpu_count()
cpu_count = 2

In [11]:
def stround(x):
    c = np.floor(x)
    return c + ((x - c) > np.random.rand())

def activate(x):
    sigma = 1/ (1 + np.exp(-x))
    return sigma

def energy(a, b, W, visible_vector):
    '''
    Enerexponentf Restricted Bolztmann machine for given parameters.
    CAUTION: a, b must be column vector. 
    '''
    #W.shape = (n_hid, 784)
    l = len(b) #n_hid
    v = visible_vector.reshape(784,1) #v.shape = (784,1)
    a, b = a.reshape(784,1), b.reshape(l,1) #both are column vector
    h = stround(activate(np.matmul(v.T,W.T).T + b)) #column vector
    e = -np.dot(a.T,v) - np.dot(b.T,h) - np.matmul(np.matmul(v.T, W.T), h) #a scalar
    
    return e

def initial(lattice_size):
    '''
    Making random configuration which consists of 0/1.
    '''
    config = np.random.randint(2, size = (lattice_size, lattice_size))
    return config

def function1(index_E1):
    global ini_index
    for k in range(len(index_E1)-1):
        c_1, c_2 = index_E1[k][1], index_E1[k+1][1]
        if e >= c_1 and e <= (c_1 + c_2)/2:
            ini_index = index_E1[k][0]
            break
        elif e >= (c_1 + c_2)/2 and e <= c_2: 
            ini_index = index_E1[k+1][0]
            break

def function2(index_E2):
    global cha_index
    for k in range(len(index_E2)-1):
        c_1, c_2 = index_E2[k][1], index_E2[k+1][1]
        if ee >= c_1 and ee <= (c_1 + c_2)/2:
            cha_index = index_E2[k][0]
            break
        elif ee >= (c_1 + c_2)/2 and ee <= c_2: 
            cha_index = index_E2[k+1][0]
            break
            
def multi_processing(index_E, function0):
    chunks = [index_E[i::cpu_count] for i in range(cpu_count)]
    pool = Pool(processes=cpu_count)
    result = pool.map_async(function0, chunks)
#     while not result.ready():
#         time.sleep(0.5)

In [12]:
def wang_landau(a,b,W): #lattice_size = 28 for MNIST RBM
    '''
    Trained model의 parameters를 넣고, sample_size만큼의 spin flip을 "시도"합니다.
    Energy space를 1간격으로 uniform하게 나눕니다.
    '''
    config = initial(28) #초기 이미지 만들기
    
    
    ######## 이 부분은 경험적으로 얻은 값을 대입하시면 됩니다.####################
    E_min=1600
    E_max=3500
    E = np.linspace(E_min, E_max, 400)
    ##############################################################################

    e = energy(a, b, W, config) #initial configuration의 energy입니다.
    ln_gE = np.zeros(len(E))
    Histogram = np.zeros(len(E))
    log_modi_factor = 1
    
    index_E1, index_E2 = [], []
    ini_index, cha_index = 0, 0
    
    count_flat, count_sweep = 0, 0

    for i in enumerate(E): 
        index_E1.append(i)
    for j in enumerate(E): 
        index_E2.append(j)
    
    while count_flat <= 27:
        count_sweep += 1
        for i in range(28):
            for j in range(28):
                s = config[i,j]

                #spin flip
                #count_flip += 1
                if s == 0:
                    s = 1
                else:
                    s = 0
                config[i,j] = s
                ee = energy(a, b, W, config) #spin을 하나 flip한 사진의 에너지를 구합니다.

#                 for k in range(len(index_E1)-1):
#                     c_1, c_2 = index_E1[k][1], index_E1[k+1][1]
#                     if e >= c_1 and e <= (c_1 + c_2)/2:
#                         ini_index = index_E1[k][0]
#                         break
#                     elif e >= (c_1 + c_2)/2 and e <= c_2: 
#                         ini_index = index_E1[k+1][0]
#                         break

#                 for k in range(len(index_E2)-1):
#                     c_1, c_2 = index_E2[k][1], index_E2[k+1][1]
#                     if ee >= c_1 and ee <= (c_1 + c_2)/2:
#                         cha_index = index_E2[k][0]
#                         break
#                     elif ee >= (c_1 + c_2)/2 and ee <= c_2: 
#                         cha_index = index_E2[k+1][0]
# #                         break
#                 multi_processing(index_E1, e)
#                 multi_processing(index_E2, ee)
                multi_processing(index_E1, function1)
                multi_processing(index_E2, function2)
                p = np.exp(ln_gE[ini_index] - ln_gE[cha_index])
                if p > np.random.rand():
                    s = s #keeps flipped value
                    Histogram[cha_index] += 1
                    ln_gE[cha_index] += log_modi_factor
                    e = ee
                else:
                    if s == 0:
                        s = 1
                    else:
                        s = 0 
                    config[i,j] = s #spin flip is rejected
                    Histogram[ini_index] += 1
                    ln_gE[ini_index] += log_modi_factor
                    e = e

        #Is the Histogram flat?
        if count_sweep % 10000 == 0:
            Havg = np.mean(Histogram)
            Hmin = min(Histogram)
            print(Histogram)
            if Havg*0.80 <= Hmin:
                Histogram = np.zeros_like(E)
                log_modi_factor *= 0.5
                count_flat += 1
                print(f"{count_flat}")
    return ln_gE

Partition function $Z = \sum_{E} g(E) e^{-E}$. <br>
Free energy is defined as $F = - \text{log}{Z}$. <br>

## 필요한 parameter를 불러오시면 됩니다.

In [13]:
#여기서 필요하신 값들을 대입하시면 됩니다.
a = pd.read_pickle('parameters/n_hid=4_vol_12000_bias_v.pkl')
b = pd.read_pickle('parameters/n_hid=4_vol_12000_bias_h.pkl')
W = pd.read_pickle('parameters/n_hid=4_vol_12000_weight.pkl')

In [14]:
start = time.time()
lngE = wang_landau(a,b,W)
end = time.time() - start
print(f"Takes {end/60} min")

  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
Process ForkPoolWorker-236:
Process ForkPoolWorker-219:
Process ForkPoolWorker-197:
Process ForkPoolWorker-204:
Process ForkPoolWorker-193:
Process ForkPoolWorker-187:
Process ForkPoolWorker-186:
Process ForkPoolWorker-189:
Process ForkPoolWorker-179:
Process ForkPoolWorker-181:
Process ForkPoolWorker-178:
Process ForkPoolWorker-161:
Process ForkPoolWorker-173:
Process ForkPoolWorker-170:
Process ForkPoolWorker-165:
Process ForkPoolWorker-166:
Process ForkPoolWorker-159:
Process ForkPoolWorker-154:
Process ForkPoolWorker-153:
Process ForkPoolWorker-135:
Process ForkPoolWorker-142:
Process ForkPoolWorker-156:
Process ForkPoolWorker-151:
Process ForkPoolWorker-137:
Process ForkPoolWorker-256:
Process ForkPoolWorker-9:
Process ForkPoolWorker-55:
Process ForkPoolWorker-57:
Process ForkPoolWorker-5:
Process ForkPoolWorker-32:
Process ForkPoolWorker-24:
Process ForkPoolWork

KeyboardInterrupt: 

  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/mu

In [ ]:
import pickle as pkl
with open('lngE_8.pkl', 'wb') as f:
    pkl.dump(lngE, f)

Exception in thread Thread-757:
Traceback (most recent call last):
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 412, in _handle_workers
    pool._maintain_pool()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 248, in _maintain_pool
    self._repopulate_pool()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 241, in _repopulate_pool
    w.start()
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 112, in start
    self._popen = self._Popen(self)
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/Users/gilhankim/opt/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line